In [2]:
import os
import glob
import random
import shutil
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras import models, layers
from tqdm import tqdm

In [3]:
# Where the text files are going to live.
dataset_path = "dataset"
dataset_path_all = os.path.join(dataset_path, "all")
dataset_path_train = os.path.join(dataset_path, "train")
dataset_path_valid = os.path.join(dataset_path, "valid")


In [4]:

# # Just use 20 files.
# file_number = 20

# # Gather the corpus if it has not been gathered yet.
# if not os.path.exists(dataset_path):

#     # Create all the folders.
#     for path in [dataset_path, dataset_path_all, dataset_path_train, dataset_path_valid]:
#         if not os.path.exists(path):
#             os.mkdir(path)

#     # Clone the repo.
#     !git clone https://github.com/vilmibm/lovecraftcorpus
        
#     # Find all the files.
#     paths_all = glob.glob("lovecraftcorpus/*.txt")
#     print(sorted(paths_all))

#     # Do not use all.
#     paths_all = paths_all[:file_number]

#     # Split 80/20.
#     split_index = int(len(paths_all) * 0.8)
#     paths_train = paths_all[:split_index]
#     paths_valid = paths_all[split_index:]

#     # Copy files.
#     def copy(paths, destination):
#         for path in paths:
#             shutil.copy2(path, destination)
#     copy(paths_all, dataset_path_all)
#     copy(paths_train, dataset_path_train)
#     copy(paths_valid, dataset_path_valid)

#     # Delete repo.
#     !del -rf lovecraftcorpus

#     # Done.
#     print("Corpus downloaded.")

In [5]:
!dir /b dataset
print("")
!dir /b dataset\all
print("")
!dir /b dataset\train
print("")
!dir /b dataset\valid

all
train
valid

alchemist.txt
arthur_jermyn.txt
azathoth.txt
beast.txt
beyond_wall_of_sleep.txt
book.txt
celephais.txt
charles_dexter_ward.txt
clergyman.txt
colour_out_of_space.txt
cool_air.txt
crawling_chaos.txt
cthulhu.txt
dagon.txt
descendent.txt
doorstep.txt
dreams_in_the_witch.txt
dunwich.txt
erich_zann.txt
ex_oblivione.txt

alchemist.txt
arthur_jermyn.txt
azathoth.txt
beast.txt
beyond_wall_of_sleep.txt
book.txt
celephais.txt
charles_dexter_ward.txt
clergyman.txt
colour_out_of_space.txt
cool_air.txt
crawling_chaos.txt
cthulhu.txt
dagon.txt
descendent.txt
doorstep.txt

dreams_in_the_witch.txt
dunwich.txt
erich_zann.txt
ex_oblivione.txt


In [6]:
batch_size = 32
seed = 42

def create_dataset(dataset_path):
    dataset = preprocessing.text_dataset_from_directory(
        dataset_path,
        labels=None,
        batch_size=batch_size,
        seed=seed
    )

    return dataset


dataset_original_all = create_dataset(dataset_path_all)
dataset_original_train = create_dataset(dataset_path_train)
dataset_original_valid = create_dataset(dataset_path_valid)

Found 20 files belonging to 1 classes.
Found 16 files belonging to 1 classes.
Found 4 files belonging to 1 classes.


In [7]:
# for x in dataset_original_all.take(1):
#     print(x)

In [8]:
vacabulary_size = 10_000


encoder = layers.TextVectorization(
    max_tokens=vacabulary_size,
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    output_mode="int"
)

encoder.adapt(dataset_original_all)


vocabulary = encoder.get_vocabulary()

print(vocabulary[:100])

['', '[UNK]', 'the', 'and', 'of', 'to', 'a', 'in', 'was', 'he', 'had', 'that', 'his', 'i', 'it', 'as', 'with', 'at', 'which', 'from', 'for', 'on', 'but', 'not', 'were', 'by', 'him', 'be', 'all', 'an', 'this', 'they', 'there', 'or', 'no', 'my', 'when', 'could', 'one', 'have', 'been', 'would', 'what', 'some', 'so', 'old', 'about', 'me', 'its', 'out', 'more', 'is', 'only', 'seemed', 'their', 'now', 'very', 'before', 'who', 'up', 'did', 'you', 'after', 'then', 'time', 'into', 'them', 'than', 'ward', 'must', 'found', 'any', 'even', 'where', 'man', 'like', 'down', 'if', 'came', 'through', 'strange', 'her', 'great', 'though', 'never', 'upon', 'those', 'saw', 'house', 'night', 'willett', 'heard', 'thing', 'room', 'over', 'other', 'whose', 'she', 'curwen', 'come']


In [12]:
sequence_length = 32

def create_dataset_for_autoregression(dataset):
      x_inputs = []
      y_inputs = []

      for books in dataset:
            books = encoder(books).numpy()
            print(books)
            break # TODO Remove me alter
      
      return tf.data.Dataset.from_tensor_slices((x_inputs, y_inputs))

dataset_train = create_dataset_for_autoregression(dataset_original_train)
dataset_valid = create_dataset_for_autoregression(dataset_original_valid)

[[   2 5452  274 ...    0    0    0]
 [   2  334    4 ...  848 5359  581]
 [   2 5194  590 ...    0    0    0]
 ...
 [ 124    2  271 ...    0    0    0]
 [7391   13  266 ...    0    0    0]
 [1923  249   61 ...    0    0    0]]
[[   2  417  160 ...   67    9   60]
 [ 225    7    2 ...    0    0    0]
 [   2 1011    4 ...    0    0    0]
 [   1    1   36 ...    0    0    0]]
